<a href="https://colab.research.google.com/github/MathBorgess/data_science_studies/blob/main/deep_learning/convolutional/basic_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
(train_set, train_labels), (test_set, test_labels) = tf.keras.datasets.cifar10.load_data()
#pixels goes from 255 to 0, so lets normalize it

train_set, test_set = train_set / 255.0, test_set / 255.0

170498071/170498071 [==============================] - 13s 0us/step


In [11]:
import numpy as np

[np.unique(train_labels), train_set.shape]

[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), (50000, 32, 32, 3)]

In [12]:
class ImageClassifier(tf.keras.Model):
    def __init__(self, input_shape, class_num, dense_units):
        super(ImageClassifier, self).__init__()
        #the filter input is the depth of the spatial convolutional layer
        self.conv_one = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu', input_shape=input_shape )
        self.pooler_one = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.conv_two = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu' )
        self.pooler_two = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.conv_two = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu' )
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(activation='relu', units=dense_units)
        self.outputer = tf.keras.layers.Dense(units=class_num)

    def call(self, inputs):
        feature_map = self.conv_one(inputs)
        reduced_feature_map = self.pooler_one(feature_map)
        feature_map = self.conv_two(reduced_feature_map)
        reduced_feature_map = self.pooler_two(feature_map)
        y = self.dense(self.flatten(reduced_feature_map))
        return self.outputer(y)

In [17]:
model = ImageClassifier(input_shape=train_set[1:], class_num=len(np.unique(train_labels)), dense_units=64)
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [14]:
model(tf.constant([train_set[0]]))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ 0.0531631 ,  0.27405676, -0.05456904,  0.05783781, -0.03702664,
         0.13045484, -0.07974085, -0.05092875,  0.06262868,  0.17407289]],
      dtype=float32)>

In [18]:
model.fit(train_set,train_labels,
          epochs=10,
          validation_data=(test_set, test_labels))

Epoch 1/10
1563/1563 [==============================] - 9s 5ms/step - loss: 1.4564 - val_loss: 1.1755
Epoch 2/10
1563/1563 [==============================] - 7s 4ms/step - loss: 1.0779 - val_loss: 0.9998
Epoch 3/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9461 - val_loss: 0.9852
Epoch 4/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8615 - val_loss: 0.9339
Epoch 5/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.7981 - val_loss: 0.9393
Epoch 6/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.7467 - val_loss: 0.8954
Epoch 7/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6973 - val_loss: 0.8721
Epoch 8/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.6550 - val_loss: 0.9283
Epoch 9/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.6183 - val_loss: 0.8933
Epoch 10/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.584